In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

In [0]:
def area(box):
    return (box[2] - box[0]) * (box[3] - box[1])


def intersection_over_union(boxes):
    assert(len(boxes) == 8)
    boxA = boxes[:4].values
    boxB = boxes[4:].values
    
    boxAArea = area(boxA)
    boxBArea = area(boxB)
    
    if (boxAArea == 0 or boxBArea == 0):
        return 0
        
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    interArea = max(0, xB - xA) * max(0, yB - yA)

    
    iou = interArea / float(boxAArea + boxBArea - interArea)
    return iou

In [45]:
from google.colab import drive
drive.mount('drive')

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [46]:
votes = pd.read_csv("drive/My Drive/sna9/train3_data.csv")
data = pd.read_csv("drive/My Drive/sna9/dataiou.csv", index_col=0)
data.head()

,Xmin,Ymin,Xmax,Ymax,Xmin_true,Ymin_true,Xmax_true,Ymax_true,iou,Xc,Yc,Xc_true,Yc_true
itemId,,,,,,,,,,,,,
5,151,914,511,1331,198,827,649,1340,0.525002,331.0,1122.5,423.5,1083.5
10,72,559,427,858,26,605,433,847,0.736209,249.5,708.5,229.5,726.0
13,46,600,621,981,233,698,622,920,0.394233,333.5,790.5,427.5,809.0
30,65,353,324,491,61,368,329,530,0.640055,194.5,422.0,195.0,449.0
32,9,279,388,884,30,536,328,813,0.360035,198.5,581.5,179.0,674.5


Добавлю центры прямоугольников:

In [0]:
data['Xc'] = data.Xmin + (data.Xmax - data.Xmin) / 2
data['Yc'] = data.Ymin + (data.Ymax - data.Ymin) / 2
data['Xc_true'] = data.Xmin_true + (data.Xmax_true - data.Xmin_true) / 2
data['Yc_true'] = data.Ymin_true + (data.Ymax_true - data.Ymin_true) / 2

In [0]:
data.sort_index().to_csv("drive/My Drive/sna9/dataiou.csv")

## Linear Regression

In [0]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

In [0]:
X = data[['Xc', 'Yc']]
y = data[['Xc_true', 'Yc_true']]
reg = LinearRegression().fit(X, y)
reg.score(X, y) # 0.9364559329721885

In [51]:
# возьму лучших
#data1 = data[data.iou > 0.5]
data1.shape # 629
X = data1[['Xc', 'Yc']]
y = data1[['Xc_true', 'Yc_true']]
reg = LinearRegression().fit(X, y)
reg.score(X, y) # 0.9454789828062106

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:420: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


0.9454789828062106

In [0]:
# with split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
reg = LinearRegression().fit(X_train, y_train)
reg.score(X_test, y_test) # 0.9266666757513186 # 0.9434987610294692

In [79]:
XX = PolynomialFeatures(3).fit_transform(X_train)
YY = PolynomialFeatures(3).fit_transform(y_train)
reg = LinearRegression().fit(XX, YY)
reg.score(PolynomialFeatures(3).fit_transform(X_test), PolynomialFeatures(3).fit_transform(y_test)) # 0.9446583024460893

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:420: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


0.9446583024460893

## Predict 

In [0]:
X = data1[['Xc', 'Yc']]

In [84]:
pred = reg.predict(PolynomialFeatures(3).fit_transform(X))
pred

array([[1.00000000e+00, 3.29701203e+02, 1.13032357e+03, ...,
        1.26914587e+08, 4.21521911e+08, 1.45410952e+09],
       [1.00000000e+00, 2.54750276e+02, 7.02129643e+02, ...,
        4.68671542e+07, 1.26473281e+08, 3.52879006e+08],
       [1.00000000e+00, 2.01699106e+02, 4.17004775e+02, ...,
        1.53581124e+07, 3.32828006e+07, 7.32194761e+07],
       ...,
       [1.00000000e+00, 3.59378462e+02, 7.49199856e+02, ...,
        9.85242481e+07, 2.00923219e+08, 4.24682940e+08],
       [1.00000000e+00, 4.44343313e+02, 1.04507132e+03, ...,
        2.08861167e+08, 4.87494222e+08, 1.16742322e+09],
       [1.00000000e+00, 1.14742114e+02, 5.83687110e+02, ...,
        8.99085156e+06, 4.12389494e+07, 2.04044183e+08]])

In [85]:
df_pred = pd.DataFrame({'Xpr': pred[:, 1], 'Ypr': pred[:, 2]}, index=data1.index)
df_pred.head()

,Xpr,Ypr
itemId,,
5,329.701203,1130.323572
10,254.750276,702.129643
30,201.699106,417.004775
64,139.796020,487.158359
78,338.157119,441.813385


In [0]:
# Восстановление координат 0.6391735466692768
df_pred['Xmin'] = df_pred.Xpr - (data.Xmax - data.Xmin) / 2
df_pred['Ymin'] = df_pred.Ypr - (data.Ymax - data.Ymin) / 2
df_pred['Xmax'] = df_pred.Xpr + (data.Xmax - data.Xmin) / 2
df_pred['Ymax'] = df_pred.Ypr + (data.Ymax - data.Ymin) / 2

In [0]:
# Восстановление координат 2 0.642070141649491
df_pred['Xmin'] = df_pred.Xpr - (data.Xmax - data.Xmin) / 2
df_pred['Ymin'] = data.Ymin
df_pred['Xmax'] = data.Xmax
df_pred['Ymax'] = df_pred.Ypr + (data.Ymax - data.Ymin) / 2

In [91]:
data["iou"] = pd.concat([df_pred[['Xmin','Ymin','Xmax','Ymax']], data[['Xmin_true','Ymin_true','Xmax_true','Ymax_true']]], axis=1).apply(intersection_over_union, axis=1)
data["iou"].mean() # 0.5618455598378245

0.6472517283428899

In [0]:
# Восстановление координат 3 СДВИГ ЦЕНТРА В СЕРЕДИНУ МЕЖДУ РЕАЛЬНЫМ И ПРЕДСКАЗАННЫМ


# TRY IT...

In [0]:
votest = pd.read_csv("drive/My Drive/sna9/test1_data.csv", index_col=0)

In [0]:
Xminmin = votest.groupby("itemId")['Xmin'].min().reset_index().set_index('itemId')
Yminmin = votest.groupby("itemId")['Ymin'].min().reset_index().set_index('itemId')
Xmaxmax = votest.groupby("itemId")['Xmax'].max().reset_index().set_index('itemId')
Ymaxmax = votest.groupby("itemId")['Ymax'].max().reset_index().set_index('itemId')

In [67]:
#maxbox = pd.concat([Xminmin,Yminmin,Xmaxmax,Ymaxmax], axis=1)
maxbox.head()

,Xmin,Ymin,Xmax,Ymax,Xc,Yc
itemId,,,,,,
18,126,634,764,1132,445.0,883.0
19,27,559,119,729,73.0,644.0
33,21,380,414,652,217.5,516.0
62,35,824,639,1262,337.0,1043.0
114,41,570,216,780,128.5,675.0


In [0]:
maxbox.sort_index().to_csv("drive/My Drive/sna9/test_max.csv")

In [0]:
maxbox['Xc'] = maxbox.Xmin + (maxbox.Xmax - maxbox.Xmin) / 2
maxbox['Yc'] = maxbox.Ymin + (maxbox.Ymax - maxbox.Ymin) / 2

In [92]:
X = maxbox[['Xc', 'Yc']]
pred = reg.predict(PolynomialFeatures(3).fit_transform(X))
pred

array([[1.00000000e+00, 4.36984146e+02, 8.74449255e+02, ...,
        1.69353967e+08, 3.34471683e+08, 6.81534278e+08],
       [1.00000000e+00, 7.69977343e+01, 6.41449238e+02, ...,
        5.16038162e+06, 3.36096146e+07, 2.66356867e+08],
       [1.00000000e+00, 2.24791165e+02, 5.09544091e+02, ...,
        2.55912684e+07, 5.89598458e+07, 1.36828936e+08],
       ...,
       [1.00000000e+00, 4.84478623e+02, 9.14002266e+02, ...,
        2.17357804e+08, 4.06800920e+08, 7.81242904e+08],
       [1.00000000e+00, 1.05944453e+02, 9.40017162e+02, ...,
        7.84872957e+06, 9.20984416e+07, 8.25010385e+08],
       [1.00000000e+00, 2.96489054e+02, 7.65651515e+02, ...,
        6.89464617e+07, 1.73661848e+08, 4.55472908e+08]])

In [95]:
#df_pred = pd.DataFrame({'Xpr': pred[:, 1], 'Ypr': pred[:, 2]}, index=maxbox.index)
df_pred.head()

,Xpr,Ypr,Xmin,Ymin,Xmax,Ymax
itemId,,,,,,
18,436.984146,874.449255,117.984146,634,764,1123.449255
19,76.997734,641.449238,30.997734,559,119,726.449238
33,224.791165,509.544091,28.291165,380,414,645.544091
62,335.711766,1046.270689,33.711766,824,639,1265.270689
114,133.435166,672.998424,45.935166,570,216,777.998424


In [0]:
df_pred['Xmin'] = df_pred.Xpr - (maxbox.Xmax - maxbox.Xmin) / 2
df_pred['Ymin'] = maxbox.Ymin
df_pred['Xmax'] = maxbox.Xmax
df_pred['Ymax'] = df_pred.Ypr + (maxbox.Ymax - maxbox.Ymin) / 2

In [0]:
maxbox[['Xmin','Ymin','Xmax','Ymax']].sort_index().to_csv("drive/My Drive/sna9/lr_poly.csv", header=None)